# Business Location Data Analysis

### Imports

In [ ]:
import pandas as pd
import requests
import time
import configparser

### Constants

In [ ]:
# Read Config file
config = configparser.ConfigParser()
config.read('config.ini')

In [ ]:
# Parameter for API call - adjust as needed
RADIUS = 5000

# Get private params from config
API_KEY = config['DETAILS']['API_KEY']
LOCATION = config['DETAILS']['LOCATION']

### Define Functions

In [ ]:
def make_request(api_key, location, radius):
    
    '''
    Makes a request to Place Details API
    Returns:
        Dataframe with up to 60 results
    Parameters:
        api_key (str): API key defined with Google Account
        location (str): Latitude and longitude values for centre of search
        radius (int): Meters around location to search
    '''
    
    # Constants
    # Admin Params - keep empty as we are using GET method
    PAYLOAD = {}
    HEADERS = {}
    # Keep output_type as json
    OUTPUT_TYPE = 'json'
    
    # Define URL for API call
    url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/{OUTPUT_TYPE}?location={location}&radius={radius}&key={api_key}'
    
    # Get first page of response
    response = requests.request('GET', url, headers=HEADERS, data=PAYLOAD)
    
    # Save into a dataframe
    json = response.json()
    df = pd.DataFrame(json['results'])
    
    # See if there is a next page of 20 results
    if 'next_page_token' in json.keys():
        
        # Wait 2 seconds to allow Google to validate next_page_token
        time.sleep(2)
        
        page_token = json['next_page_token']
        url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/{OUTPUT_TYPE}?location={location}&radius={radius}&pagetoken={page_token}&key={api_key}'
        
        response = requests.request('GET', url, headers=HEADERS, data=PAYLOAD)
        
        json = response.json()
        df_2 = pd.DataFrame(json['results'])
        
        if 'next_page_token' in json.keys():
            
            time.sleep(2)
        
            page_token = json['next_page_token']
            url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/{OUTPUT_TYPE}?location={LOCATION}&radius={RADIUS}&pagetoken={page_token}&key={API_KEY}'

            response = requests.request('GET', url, headers=HEADERS, data=PAYLOAD)

            json = response.json()
            df_3 = pd.DataFrame(json['results'])
    
    # Join all dataframes and return result
    all_df = pd.concat([df, df_2, df_3])
    
    return all_df

In [ ]:
results = make_request(API_KEY, LOCATION, RADIUS)

In [ ]:
results

## Clean Data
1. Check for duplicates & reset index
2. Seperate lat and long cols
3. One Hot Encode type?


In [ ]:
len(results)

In [ ]:
results['name'].duplicated()

## Analysis
- Count by type
- Visualise on map by type
- Identify categories of interest
- Ratings & number of reviews of those categories
- Visualise by type where category of interest
- Use Place Details Atmosphere to get more details on locations of interest
- Re-run for different radiuses and locations that customers may come from - stretham, haddenham, witchford, cottenham